In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import scipy
import sys
sys.path.append("..")
import os
pd.set_option("max_columns", 10000)

%pylab inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import json
from IPython import display

from tqdm import tqdm, tqdm_notebook, tqdm_pandas
tqdm.pandas()

import copy

from Scripts.saving import submit

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from datetime import datetime


from Scripts.utils import *

import json

import featuretools as ft

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

Populating the interactive namespace from numpy and matplotlib


In [2]:
base_dir = ".."
input_dir = open( os.path.join(base_dir, "datadir.txt"), "r").read()[:-1]
pred_dir = os.path.join(base_dir, "predictions")
processed_dir = os.path.join(input_dir, "processed")
generate_dir = os.path.join(input_dir, "generate")
!tree {input_dir}

/home/furfa/work/ai-academy2019/input
├── ai-academy-2019-skill-prediction-data-csv-v1.zip
├── ai-academy-2019-skill-prediction-data-jsonlines.zip
├── dota2_abilities.csv
├── dota2_heroes.csv
├── dota2_items.csv
├── dota2_skill_test.csv
├── dota2_skill_test.jsonlines
├── dota2_skill_train.csv
├── dota2_skill_train.jsonlines
├── generate
│   ├── test_ability_upgrades[9.4]1:34.csv
│   ├── test_damage_targets[9.4]1:50.csv
│   ├── test_timeseries_no_zeros[8.4]17:49.csv
│   ├── test_timeseries_no_zeros[8.4]18:3.csv
│   ├── test_timeseries_no_zeros[9.4]1:31.csv
│   ├── train_ability_upgrades[9.4]1:34.csv
│   ├── train_damage_targets[9.4]1:50.csv
│   ├── train_timeseries_no_zeros[8.4]17:49.csv
│   ├── train_timeseries_no_zeros[8.4]18:3.csv
│   └── train_timeseries_no_zeros[9.4]1:31.csv
└── processed
    ├── heroes_proc.csv
    ├── items_proc.csv
    ├── test_ability_upgrades.csv
    ├── test_all_JSON.csv
    ├── test_damage_targets.csv
    ├── test_df_0.814.csv
    ├── test_item_purchase_log.

In [3]:
y_train = pd.read_csv( os.path.join(input_dir,"processed", 'train_all_JSON.csv'), index_col=0 ).skilled

In [4]:
train_timeseries_no_zeros = pd.read_csv(
        os.path.join(generate_dir, "train_timeseries_no_zeros[9.4]1:31.csv"), index_col=0
    )

train_ability_upgrades = pd.read_csv(
        os.path.join(generate_dir, "train_ability_upgrades[9.4]1:34.csv"), index_col=0
    )
train_damage_targets = pd.read_csv(
        os.path.join(generate_dir, "train_damage_targets[9.4]1:50.csv"), index_col=0
    )
train_damage_targets_raw = pd.read_csv(
        os.path.join(processed_dir, "train_damage_targets.csv"), index_col=0
    )
train_team = pd.read_csv(
        os.path.join(processed_dir, "train_team.csv"), index_col=0
    )
train_purchase = pd.read_csv(
        os.path.join(processed_dir, "train_item_purchase_log.csv"), index_col=0
    )

In [5]:
train = pd.read_csv(
    os.path.join(processed_dir, "train_df_0.814.csv"), index_col=0
)
train = pd.concat([
    train,
    train_timeseries_no_zeros,
    train_ability_upgrades,
    train_damage_targets_raw,
    train_team,
    train_purchase
], axis=1)

In [6]:
test_timeseries_no_zeros = pd.read_csv(
        os.path.join(generate_dir, "test_timeseries_no_zeros[9.4]1:31.csv"), index_col=0
    )

test_ability_upgrades = pd.read_csv(
        os.path.join(generate_dir, "test_ability_upgrades[9.4]1:34.csv"), index_col=0
    )
test_damage_targets = pd.read_csv(
        os.path.join(generate_dir, "test_damage_targets[9.4]1:50.csv"), index_col=0
    )
test_damage_targets_raw = pd.read_csv(
        os.path.join(processed_dir, "test_damage_targets.csv"), index_col=0
    )
test_team = pd.read_csv(
        os.path.join(processed_dir, "test_team.csv"), index_col=0
    )
test_purchase = pd.read_csv(
        os.path.join(processed_dir, "test_item_purchase_log.csv"), index_col=0
    )

In [7]:
test = pd.read_csv(
    os.path.join(processed_dir, "test_df_0.814.csv"), index_col=0
)
test = pd.concat([
    test,
    test_timeseries_no_zeros,
    test_ability_upgrades,
    test_damage_targets_raw,
    test_team,
    test_purchase
], axis=1)

In [8]:
train.sample().shape

(1, 1943)

In [9]:
counts = test.describe().T.sort_values( by="count" )
useles_on_test_columns = counts[counts["count"] < 25000].index
train.drop(useles_on_test_columns, axis=1, inplace=True)
test.drop(useles_on_test_columns, axis=1, inplace=True)

In [10]:
counts = train.describe().T.sort_values( by="count" )
useles_on_train_columns = counts[counts["count"] < 50000].index
train.drop(useles_on_train_columns, axis=1, inplace=True)
test.drop(useles_on_train_columns, axis=1, inplace=True)

In [11]:
train.shape, test.shape

((99871, 1898), (43265, 1898))

In [35]:
from sklearn.model_selection import cross_val_score
import catboost as cb

score = cross_val_score(
    cb.CatBoostClassifier(logging_level="Silent"),
    train,
    y_train,
    scoring="accuracy",
    cv = 4,
    verbose=0,
    n_jobs=-1
)
print(score, np.mean(score), np.std(score))

[0.83547599 0.83647068 0.83510234 0.83634397] 0.8358482454573745 0.0005762228119320871


In [ ]:
from sklearn.model_selection import cross_val_score
import catboost as cb

score = cross_val_score(
    cb.CatBoostClassifier(logging_level="Silent"),
    train,
    y_train,
    scoring="accuracy",
    cv = 4,
    verbose=0,
    n_jobs=-1
)
print(score, np.mean(score), np.std(score))

In [29]:
train.shape, test.shape, all(train.columns == test.columns)

((99871, 1898), (43265, 1898), False)

In [30]:
train.describe().T.sort_values( by="count" ).head()

,count,mean,std,min,25%,50%,75%,max
No Target / Point Target,53065.0,inf,NaN,0.0,0.000000,0.0,0.75,inf
Point Target / No Target,53065.0,inf,NaN,0.0,1.333333,NaN,NaN,inf
Hidden / No Target,53881.0,inf,NaN,0.0,0.000000,NaN,NaN,inf
No Target / Hidden,53881.0,inf,NaN,0.0,0.000000,0.0,NaN,inf
Point Target / Unit Target,53943.0,inf,NaN,0.0,inf,NaN,inf,inf


In [31]:
model = cb.CatBoostClassifier(logging_level='Verbose').fit(
    train, 
    y_train)

y_pred = model.predict(
    test
)

y_pred = pd.DataFrame({'id' : test.index, 'skilled': y_pred})
submit(y_pred, "cb_raw_best_df")

Learning rate set to 0.061655
0:	learn: 0.6772678	total: 220ms	remaining: 3m 39s
1:	learn: 0.6633467	total: 410ms	remaining: 3m 24s
2:	learn: 0.6501461	total: 614ms	remaining: 3m 24s
3:	learn: 0.6386066	total: 820ms	remaining: 3m 24s
4:	learn: 0.6283531	total: 1.02s	remaining: 3m 22s
5:	learn: 0.6186804	total: 1.21s	remaining: 3m 20s
6:	learn: 0.6105189	total: 1.41s	remaining: 3m 20s
7:	learn: 0.6018398	total: 1.61s	remaining: 3m 20s
8:	learn: 0.5950267	total: 1.8s	remaining: 3m 18s
9:	learn: 0.5884380	total: 2s	remaining: 3m 17s
10:	learn: 0.5825266	total: 2.2s	remaining: 3m 17s
11:	learn: 0.5772264	total: 2.39s	remaining: 3m 17s
12:	learn: 0.5714561	total: 2.59s	remaining: 3m 16s
13:	learn: 0.5666047	total: 2.78s	remaining: 3m 15s
14:	learn: 0.5619710	total: 2.98s	remaining: 3m 15s
15:	learn: 0.5574646	total: 3.17s	remaining: 3m 14s
16:	learn: 0.5536775	total: 3.37s	remaining: 3m 15s
17:	learn: 0.5500720	total: 3.56s	remaining: 3m 14s
18:	learn: 0.5460767	total: 3.75s	remaining: 3m 1